In [1]:
# File management
import glob
import os
import shutil
import csv

# Data processing
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
from skimage import io
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit, minimize
from scipy.ndimage import gaussian_filter1d
from scipy.stats import norm
from PIL import Image, ImageEnhance, ImageOps

# Utilities
import multiprocessing as mp
from multiprocessing import Pool, cpu_count
mp.set_start_method('fork', force=True)
from ipywidgets import interact, FloatSlider, Layout, interactive
import random
from tqdm import tqdm
import itertools
import cv2
from natsort import natsorted

# Set up logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
def prepare_conditions(data_path, num_reps):
    # List conditions while ignoring 'output_data'
    conditions = natsorted([
        f for f in os.listdir(data_path) 
        if os.path.isdir(os.path.join(data_path, f)) and f != 'output_data'
    ])
    
    # Generate subconditions list based on num_reps
    subconditions = [f"Rep{x}" for x in range(1, num_reps + 1)]
    
    return conditions, subconditions

# Example usage
calibration_curve_paths = sorted(glob.glob("../../../../Thomson Lab Dropbox/SURF_activedrops/Edgar/calibration_curve/***ugml.tif"))

data_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/"
conditions, subconditions = prepare_conditions(data_path, 3)

print("Conditions:", conditions)
print("Subconditions:", subconditions)


Conditions: ['kif3']
Subconditions: ['Rep1', 'Rep2', 'Rep3']


In [3]:
def reorgTiffsToOriginal(data_path, conditions, subconditions):
    """
    Args:
        data_path (str): Path to the data directory.
        conditions (list): List of conditions.
        subconditions (list): List of subconditions.
        
    This function renames the subconditions as PosX and moves the raw data to the "original" folder.
    """
    for condition in conditions:
        # Get the actual subconditions in the directory
        actual_subconditions = [name for name in os.listdir(os.path.join(data_path, condition)) if os.path.isdir(os.path.join(data_path, condition, name))]
        
        # Rename the actual subconditions to match the subconditions in your list
        for i, actual_subcondition in enumerate(sorted(actual_subconditions)):
            os.rename(os.path.join(data_path, condition, actual_subcondition), os.path.join(data_path, condition, subconditions[i]))
        
        for subcondition in subconditions:
            # Construct the path to the subcondition directory
            subcondition_path = os.path.join(data_path, condition, subcondition)
            
            # Create the path for the "original" directory within the subcondition directory
            original_dir_path = os.path.join(subcondition_path, "original")
            
            # Always create the "original" directory
            os.makedirs(original_dir_path, exist_ok=True)
            
            # Iterate over all files in the subcondition directory
            for filename in os.listdir(subcondition_path):
                # Check if the file is a .tif file
                if filename.endswith(".tif"):
                    # Construct the full path to the file
                    file_path = os.path.join(subcondition_path, filename)
                    
                    # Construct the path to move the file to
                    destination_path = os.path.join(original_dir_path, filename)
                    
                    # Move the file to the "original" directory
                    shutil.move(file_path, destination_path)
            print(f"Moved .tif files from {subcondition_path} to {original_dir_path}")


def ensure_output_dir(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)


def calculate_mean_intensity(path):
    """Calculate mean intensity of an image."""
    return io.imread(path).mean()


def calculate_protein_concentration(mean_intensity, intercept, slope):
    """Calculate protein concentration in ng/ul and nM."""
    conc_ng_ul = (mean_intensity - intercept) / slope
    return conc_ng_ul


def calculate_protein_concentration_nM(conc_ng_ul, mw_kda):
    """Convert protein concentration from ng/ul to nM."""
    conc_nM = (conc_ng_ul * 1e-3) / (mw_kda * 1e3) * 1e9
    return conc_nM


def calculate_number_of_protein_molecules(protein_mass, mw_kda):
    """Calculate number of protein molecules."""
    return (protein_mass * 6e14) / (mw_kda * 1e3)


def convert_time_units(time_values_s):
    """Convert time values from seconds to minutes and hours."""
    time_values_min = time_values_s / 60
    time_values_h = time_values_s / 3600
    return time_values_s, time_values_min, time_values_h


def process_image(args):
    image_file, output_directory_path, channel, slope, intercept, vmax, time_interval, i, show_scalebar, min_frame, skip_frames, condition, subcondition = args
    # Read the image into a numpy array
    intensity_matrix = io.imread(image_file)

    if channel == "cy5":
        matrix_to_plot = intensity_matrix
        # Use raw intensity for cy5 channel
        label = 'Fluorescence Intensity'
    else:
        # Convert intensity values to protein concentration using the calibration curve
        matrix_to_plot = calculate_protein_concentration(intensity_matrix, slope, intercept)
        matrix_to_plot = matrix_to_plot / 27000 * 1E6
        label = 'Protein concentration (nM)'

    # Plot the heatmap
    fig, ax = plt.subplots(figsize=(12, 12))
    im = ax.imshow(matrix_to_plot, cmap='gray', interpolation='nearest', vmin=0, vmax=vmax)

    if show_scalebar:
        plt.colorbar(im, ax=ax, label=label)
    plt.title(f"Time (min): {(i - min_frame) * time_interval * skip_frames / 60:.2f} \nTime (h): {(i - min_frame) * time_interval * skip_frames / 3600:.2f} \n{condition} - {subcondition} - {channel}", fontsize=14)
    plt.xlabel('x [µm]')
    plt.ylabel('y [µm]')
    plt.grid(True, color='#d3d3d3', linewidth=0.5, alpha=0.5)

    # Save the heatmap
    heatmap_filename = f"heatmap_frame_{i}.png"
    heatmap_path = os.path.join(output_directory_path, heatmap_filename)
    plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0.1, dpi=300)
    plt.close(fig)


def fluorescence_heatmap(data_path, conditions, subconditions, channel, time_interval_list, min_frame, max_frame, vmax, skip_frames=1, calibration_curve_paths=None, show_scalebar=True):
    """
    Reads each image as a matrix, creates, and saves a heatmap representing the normalized pixel-wise fluorescence intensity.

    Args:
    - data_path (str): Base directory where the images are stored.
    - conditions (list): List of conditions defining subdirectories within the data path.
    - subconditions (list): List of subconditions defining further subdirectories.
    - channel (str): Channel specifying the fluorescence ('cy5' or 'gfp').
    - time_interval_list (list): List of time intervals in seconds between frames for each condition.
    - min_frame (int): Minimum frame number to start processing from.
    - max_frame (int): Maximum frame number to stop processing at.
    - vmax (float): Maximum value for color scale in the heatmap.
    - skip_frames (int): Interval to skip frames (default is 1, meaning process every frame).
    - calibration_curve_paths (list): List of file paths for the calibration curve images.
    - show_scalebar (bool): Whether to show the color scale bar in the heatmap.
    """
    output_data_dir = os.path.join(data_path, "output_data", "movies")
    ensure_output_dir(output_data_dir)

    def setup_calibration_curve(channel, calibration_curve_paths):
        if channel != "cy5":
            # Calibration curve data and fit
            sample_concentration_values = [0, 2, 5, 10, 20, 40, 80, 160, 320]

            if calibration_curve_paths is None or len(calibration_curve_paths) != len(sample_concentration_values):
                raise ValueError(f"Mismatch in lengths: {len(calibration_curve_paths)} calibration images, {len(sample_concentration_values)} sample concentrations")

            with mp.Pool(mp.cpu_count()) as pool:
                mean_intensity_calibration = pool.map(calculate_mean_intensity, calibration_curve_paths)
            return np.polyfit(sample_concentration_values, mean_intensity_calibration, 1)
        return None, None

    for idx, condition in enumerate(conditions):
        time_interval = time_interval_list[idx]

        for subcondition in subconditions:
            # Determine the directory paths based on the channel
            input_directory_path = os.path.join(data_path, condition, subcondition, "original")
            output_directory_path = os.path.join(output_data_dir, f"{condition}_{subcondition}_heatmaps_{channel}")

            # Create the output directory if it doesn't exist, or clear it if it does
            if os.path.exists(output_directory_path):
                shutil.rmtree(output_directory_path)
            os.makedirs(output_directory_path, exist_ok=True)

            # Get all .tif files in the folder
            image_files = sorted(glob.glob(os.path.join(input_directory_path, f"*{channel}*.tif")))[min_frame:max_frame:skip_frames]

            # Setup calibration curve for non-cy5 channels
            slope, intercept = setup_calibration_curve(channel, calibration_curve_paths)

            # Prepare arguments for multiprocessing
            args = [(image_file, output_directory_path, channel, slope, intercept, vmax, time_interval, i, show_scalebar, min_frame, skip_frames, condition, subcondition) for i, image_file in enumerate(image_files, start=min_frame)]

            # Use multiprocessing to process images
            with mp.Pool(mp.cpu_count()) as pool:
                list(tqdm(pool.imap(process_image, args), total=len(args), desc=f"Processing {condition} - {subcondition}"))


def prepare_conditions(data_path, num_reps):
    # List conditions while ignoring 'output_data'
    conditions = natsorted([
        f for f in os.listdir(data_path) 
        if os.path.isdir(os.path.join(data_path, f)) and f != 'output_data'
    ])
    
    # Generate subconditions list based on num_reps
    subconditions = [f"Rep{x}" for x in range(1, num_reps + 1)]
    
    return conditions, subconditions


def process_video_creation(args):
    condition, subcondition, images_dir, out_path, frame_rate, max_frame = args

    image_files = natsorted(glob.glob(os.path.join(images_dir, "*.png")))

    if not image_files:
        print(f"No images found for subcondition {subcondition}.")
        return

    # Limit the number of files if max_frame is specified
    image_files = image_files[:max_frame] if max_frame is not None else image_files

    # Get the resolution of the first image (assuming all images are the same size)
    first_image = cv2.imread(image_files[0])
    video_resolution = (first_image.shape[1], first_image.shape[0])  # Width x Height

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(out_path, fourcc, frame_rate, video_resolution)

    for file in tqdm(image_files, desc=f"Creating video for {condition}", leave=False):
        img = cv2.imread(file)
        out.write(img)  # Write the image as a frame in the video

    out.release()
    print(f"Video saved to {out_path}")


def create_movies(data_path, conditions, subconditions, channel, frame_rate=30, max_frame=None):
    """
    Creates video files from heatmaps stored in the specified directory.

    Args:
    - data_path (str): Base path where the heatmaps are stored.
    - conditions (list): List of conditions defining subdirectories within the data path.
    - subconditions (list): List of subconditions defining further subdirectories.
    - channel (str): The specific channel being processed ('cy5' or 'gfp').
    - frame_rate (int): Frame rate for the output video. Defaults to 30.
    - max_frame (int, optional): Maximum number of frames to be included in the video. If None, all frames are included.
    """
    output_data_dir = os.path.join(data_path, "output_data", "movies")

    # Prepare arguments for multiprocessing
    args_list = []
    for condition in conditions:
        for subcondition in subconditions:
            images_dir = os.path.join(output_data_dir, f"{condition}_{subcondition}_heatmaps_{channel}")
            video_filename = f"{condition}_{subcondition}_{channel}.avi"
            out_path = os.path.join(output_data_dir, video_filename)
            args_list.append((condition, subcondition, images_dir, out_path, frame_rate, max_frame))

    # Use multiprocessing to process video creation for subconditions of a single condition
    with mp.Pool(mp.cpu_count()) as pool:
        list(tqdm(pool.imap(process_video_creation, args_list), total=len(args_list), desc="Creating videos", leave=True))


def process_intensity(path, slope, intercept, mw_kda):
    mean_intensity = calculate_mean_intensity(path)
    protein_concentration_ng_ul = calculate_protein_concentration(mean_intensity, intercept, slope)
    protein_concentration_nM = calculate_protein_concentration_nM(protein_concentration_ng_ul, mw_kda)
    return mean_intensity, protein_concentration_ng_ul, protein_concentration_nM


def quantify_tiffiles(data_path, conditions, subconditions, calibration_curve_paths, mw_kda_list, droplet_volume_list, time_interval_s_list):
    """Process images to calculate protein concentration and generate plots."""
    all_data = []

    # Sort the calibration curve paths
    calibration_curve_paths = sorted(calibration_curve_paths)

    # Calibration curve data and fit
    sample_concentration_values = [0, 2, 5, 10, 20, 40, 80, 160, 320]
    with mp.Pool(mp.cpu_count()) as pool:
        mean_intensity_calibration = pool.map(calculate_mean_intensity, calibration_curve_paths)
    slope, intercept = np.polyfit(sample_concentration_values, mean_intensity_calibration, 1)

    for idx, condition in enumerate(conditions):
        # Get condition-specific parameters
        mw_kda = mw_kda_list[idx]
        droplet_volume = droplet_volume_list[idx]
        time_interval_s = time_interval_s_list[idx]

        for subcondition in subconditions:
            # Construct paths based on condition and subcondition
            pattern = os.path.join(data_path, condition, subcondition, "original", "img_*********_gfp-4x_000.tif")
            paths = sorted(glob.glob(pattern))

            if not paths:
                print(f"No image files found for condition {condition}, subcondition {subcondition}.")
                continue

            # Calculate mean intensity, protein concentrations in ng/ul, and nM in parallel
            with mp.Pool(mp.cpu_count()) as pool:
                results = list(tqdm(pool.imap(lambda path: process_intensity(path, slope, intercept, mw_kda), paths), total=len(paths), desc=f"Processing {condition} - {subcondition}"))

            mean_intensity_list, protein_concentration_list, protein_concentration_nM_list = zip(*results)

            # Normalize intensities and concentrations
            min_intensity = min(mean_intensity_list)
            mean_intensity_list = np.array(mean_intensity_list) - min_intensity
            protein_concentration_list = np.array(protein_concentration_list) - min(protein_concentration_list)
            protein_concentration_nM_list = np.array(protein_concentration_nM_list) - min(protein_concentration_nM_list)

            # Time values
            time_values_s = np.arange(len(mean_intensity_list)) * time_interval_s
            time_values_s, time_values_min, time_values_h = convert_time_units(time_values_s)
            
            df = pd.DataFrame({
                "Condition": condition,
                "Subcondition": subcondition,
                "Time_s": time_values_s,
                "Time_min": time_values_min,
                "Time_h": time_values_h,
                "Mean Intensity": mean_intensity_list,
                "Protein Concentration_ng_ul": protein_concentration_list,
                "Protein Concentration_nM": protein_concentration_nM_list
            })

            # Calculate number of protein molecules
            protein_mass_list = df["Protein Concentration_ng_ul"] * droplet_volume
            df["Number of Protein Molecules"] = [calculate_number_of_protein_molecules(mass, mw_kda) for mass in protein_mass_list]

            # Calculate rate of change of protein molecules
            t_vals = np.linspace(0, (len(df) - 1) * time_interval_s, len(df))
            dp_dt = gaussian_filter1d(np.gradient(df["Number of Protein Molecules"], t_vals), sigma=2)
            df["Rate of Change of Number of Protein Molecules (PM/s)"] = dp_dt

            # Append the data for this condition and subcondition to the list
            all_data.append(df)

    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    # Calculate mean for each condition across subconditions
    mean_df = combined_df.groupby(["Condition", "Time_s", "Time_min", "Time_h"]).mean(numeric_only=True).reset_index()

    # Set the output directory within the data path
    output_dir = os.path.join(data_path, "output_data")
    ensure_output_dir(output_dir)

    # Save combined results to CSV
    combined_csv_path = os.path.join(output_dir, "combined_experiment.csv")
    combined_df.to_csv(combined_csv_path, index=False)

    # Save mean results to CSV
    mean_csv_path = os.path.join(output_dir, "mean_experiment.csv")
    mean_df.to_csv(mean_csv_path, index=False)

    # Plotting
    plot_results(combined_df, mean_df, output_dir, sample_concentration_values, mean_intensity_calibration, slope, intercept)

    return combined_csv_path, mean_csv_path


def plot_results(df, mean_df, output_dir, sample_concentration_values, mean_intensity_calibration, slope, intercept):
    """Generate plots based on the processed data."""
    # Create subdirectories for plots
    single_plot_dir = os.path.join(output_dir, "experiment_plots", "single_plots")
    combined_plot_dir = os.path.join(output_dir, "experiment_plots", "combined_plots")
    mean_plot_dir = os.path.join(output_dir, "experiment_plots", "mean_plots")
    ensure_output_dir(single_plot_dir)
    ensure_output_dir(combined_plot_dir)
    ensure_output_dir(mean_plot_dir)

    # Plot calibration curve
    plt.figure(figsize=(10, 6))
    plt.plot(sample_concentration_values, mean_intensity_calibration, 'o', label='Data points', linewidth=0.75, markersize=5)
    plt.plot(sample_concentration_values, slope * np.array(sample_concentration_values) + intercept, 'r-', label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', linewidth=0.75)
    plt.title('Mean Intensity vs Protein Concentration')
    plt.xlabel('Protein Concentration (ug/ml)')
    plt.ylabel('Mean Intensity')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(single_plot_dir, 'mean_intensity_vs_protein_concentration.png'))
    plt.close()

    # Plot calibration curve (log scale)
    plt.figure(figsize=(10, 6))
    plt.plot(sample_concentration_values, mean_intensity_calibration, 'o', label='Data points', linewidth=0.75, markersize=5)
    plt.plot(sample_concentration_values, slope * np.array(sample_concentration_values) + intercept, 'r-', label=f'Fit: y = {slope:.2f}x + {intercept:.2f}', linewidth=0.75)
    plt.title('Mean Intensity vs Protein Concentration (Log Scale)')
    plt.xlabel('Protein Concentration (ug/ml)')
    plt.ylabel('Mean Intensity')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(single_plot_dir, 'mean_intensity_vs_protein_concentration_log.png'))
    plt.close()

    # Time units and concentration units to plot
    time_units = [("Time_s", "Time_s"), ("Time_min", "Time_min"), ("Time_h", "Time_h")]
    protein_concentration_units = [
        ("Protein Concentration_ng_ul", "Protein Concentration_ng_ul"),
        ("Protein Concentration_nM", "Protein Concentration_nM"),
        ("Number of Protein Molecules", "Number of Protein Molecules")
    ]

    # Plot protein concentration over time for each time and concentration unit
    for time_unit, time_label in time_units:
        for conc_unit, conc_label in protein_concentration_units:
            # Individual plots for each condition and subcondition
            for condition in df["Condition"].unique():
                # Create a directory for each condition's single plots
                condition_single_plot_dir = os.path.join(single_plot_dir, f"{condition}_single_plots")
                ensure_output_dir(condition_single_plot_dir)

                for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
                    condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
                    plt.figure(figsize=(10, 6))
                    plt.plot(condition_data[time_unit], condition_data[conc_unit], 'o-', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
                    plt.title(f'{conc_label} vs {time_label} for {condition} {subcondition}')
                    plt.xlabel(time_label)
                    plt.ylabel(conc_label)
                    plt.legend()
                    plt.grid(True)
                    plt.savefig(os.path.join(condition_single_plot_dir, f'{condition}_{subcondition}_{conc_label}_vs_{time_label}.png'))
                    plt.close()

            # Combined plots for all conditions and subconditions
            plt.figure(figsize=(10, 6))
            for condition in df["Condition"].unique():
                for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
                    condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
                    plt.plot(condition_data[time_unit], condition_data[conc_unit], 'o-', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
            plt.title(f'Combined {conc_label} vs {time_label} for all conditions')
            plt.xlabel(time_label)
            plt.ylabel(conc_label)
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(combined_plot_dir, f'combined_{conc_label}_vs_{time_label}.png'))
            plt.close()

            # Combined plots for all conditions and subconditions (log scale)
            plt.figure(figsize=(10, 6))
            for condition in df["Condition"].unique():
                for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
                    condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
                    plt.plot(condition_data[time_unit], condition_data[conc_unit], 'o-', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
            plt.title(f'Combined {conc_label} vs {time_label} for all conditions (Log Scale)')
            plt.xlabel(time_label)
            plt.ylabel(conc_label)
            plt.yscale('log')
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(combined_plot_dir, f'combined_{conc_label}_vs_{time_label}_log.png'))
            plt.close()

            # Mean plots for each condition
            plt.figure(figsize=(10, 6))
            for condition in mean_df["Condition"].unique():
                condition_mean_data = mean_df[mean_df["Condition"] == condition]
                plt.plot(condition_mean_data[time_unit], condition_mean_data[conc_unit], 'o-', label=f'{condition} Mean', linewidth=0.75, markersize=5)
            plt.title(f'Mean {conc_label} vs {time_label} for each condition')
            plt.xlabel(time_label)
            plt.ylabel(conc_label)
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(mean_plot_dir, f'mean_{conc_label}_vs_{time_label}.png'))
            plt.close()

            # Mean plots for each condition (log scale)
            plt.figure(figsize=(10, 6))
            for condition in mean_df["Condition"].unique():
                condition_mean_data = mean_df[mean_df["Condition"] == condition]
                plt.plot(condition_mean_data[time_unit], condition_mean_data[conc_unit], 'o-', label=f'{condition} Mean', linewidth=0.75, markersize=5)
            plt.title(f'Mean {conc_label} vs {time_label} for each condition (Log Scale)')
            plt.xlabel(time_label)
            plt.ylabel(conc_label)
            plt.yscale('log')
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(mean_plot_dir, f'mean_{conc_label}_vs_{time_label}_log.png'))
            plt.close()

    # Plot rate of change of protein molecules
    plt.figure(figsize=(10, 6))
    for condition in df["Condition"].unique():
        for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
            condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
            plt.plot(condition_data["Time_h"], condition_data["Rate of Change of Number of Protein Molecules (PM/s)"], 'o', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
    plt.title('Rate of Change of Number of Protein Molecules vs Time_h')
    plt.xlabel('Time_h')
    plt.ylabel('Rate of Change (PM/s)')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(combined_plot_dir, 'rate_of_change_of_protein_molecules_vs_time_h.png'))
    plt.close()

    # Plot rate of change of protein molecules (log scale)
    plt.figure(figsize=(10, 6))
    for condition in df["Condition"].unique():
        for subcondition in df[df["Condition"] == condition]["Subcondition"].unique():
            condition_data = df[(df["Condition"] == condition) & (df["Subcondition"] == subcondition)]
            plt.plot(condition_data["Time_h"], condition_data["Rate of Change of Number of Protein Molecules (PM/s)"], 'o', label=f'{condition} {subcondition}', linewidth=0.75, markersize=5)
    plt.title('Rate of Change of Number of Protein Molecules vs Time_h (Log Scale)')
    plt.xlabel('Time_h')
    plt.ylabel('Rate of Change (PM/s)')
    plt.yscale('log')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(combined_plot_dir, 'rate_of_change_of_protein_molecules_vs_time_h_log.png'))
    plt.close()


In [4]:
# Example usage

channel = "cy5"
time_interval_list = [120] * len(conditions)  # time intervals in seconds between frames for each condition
min_frame = 0
max_frame = None
vmax = None  # Set vmax based on your data's expected concentration range
skip_frames = 4

# Call the function
fluorescence_heatmap(data_path, conditions, subconditions, channel, time_interval_list, min_frame, max_frame, vmax, skip_frames, calibration_curve_paths, show_scalebar=False)

frame_rate = 15  # frames per second

create_movies(data_path, conditions, subconditions, channel, frame_rate=frame_rate)



# Example usage

channel = "gfp"
time_interval_list = [120] * len(conditions)  # time intervals in seconds between frames for each condition
min_frame = 0
max_frame = None
vmax = None  # Set vmax based on your data's expected concentration range

# Call the function
fluorescence_heatmap(data_path, conditions, subconditions, channel, time_interval_list, min_frame, max_frame, vmax, skip_frames, calibration_curve_paths, show_scalebar=False)

frame_rate = 15  # frames per second

create_movies(data_path, conditions, subconditions, channel, frame_rate=frame_rate)

Processing kif3 - Rep3: 100%|██████████| 9/9 [00:01<00:00,  5.03it/s]


Video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/output_data/movies/kif3_Rep1_cy5.avi


Video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/output_data/movies/kif3_Rep2_cy5.avi


Video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/output_data/movies/kif3_Rep3_cy5.avi


Processing kif3 - Rep3: 100%|██████████| 180/180 [00:27<00:00,  6.50it/s]


Video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/output_data/movies/kif3_Rep2_gfp.avi


Video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/output_data/movies/kif3_Rep3_gfp.avi


Video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/output_data/movies/kif3_Rep1_gfp.avi


Creating videos: 100%|██████████| 3/3 [00:36<00:00, 12.12s/it]


# PIV

In [5]:

# Convert a single image (helper function for multiprocessing)
def process_single_image(args):
    file_name, output_dir, brightness_factor, contrast_factor, num_digits, i = args
    image = Image.open(file_name).convert("L")
    image_resized = image.resize((2048, 2048), Image.LANCZOS)

    enhancer = ImageEnhance.Brightness(image_resized)
    image_brightened = enhancer.enhance(brightness_factor)
    enhancer = ImageEnhance.Contrast(image_brightened)
    image_contrasted = enhancer.enhance(contrast_factor)

    padded_index = str(i + 1).zfill(num_digits)
    base_file_name = f'converted_image_{padded_index}.tif'
    processed_image_path = os.path.join(output_dir, base_file_name)
    image_contrasted.save(processed_image_path, format='TIFF', compression='tiff_lzw')

# Convert PIVlab images to the right size using multiprocessing
def convert_images(data_path, conditions, subconditions, max_frame, brightness_factor=1, contrast_factor=1, skip_frames=1):
    """
    Converts, resizes, and adjusts the brightness and contrast of images for multiple conditions and 
    subconditions, then saves the processed images in new directories.

    Args:
    - data_path (str): Base directory where the original PIV movie images are stored.
    - conditions (list of str): List of conditions defining subdirectories within the data path.
    - subconditions (list of str): List of subconditions defining sub-subdirectories within each condition directory.
    - max_frame (int, optional): Maximum number of images to process. If None, all images in the directory are processed.
    - brightness_factor (float, optional): Factor to adjust the brightness of the images. Defaults to 1 (no change).
    - contrast_factor (float, optional): Factor to adjust the contrast of the images. Defaults to 1 (no change).
    - skip_frames (int, optional): Number of frames to skip between processing. Defaults to 1 (no skipping).
    """
    for condition in conditions:
        for subcondition in subconditions:
            input_dir = os.path.join(data_path, condition, subcondition, "piv_movie")
            output_dir = os.path.join(data_path, condition, subcondition, "piv_movie_converted")

            os.makedirs(output_dir, exist_ok=True)

            input_files = natsorted(glob.glob(os.path.join(input_dir, '*.jpg')))

            if max_frame:
                input_files = input_files[:max_frame]

            # Apply frame skipping
            input_files = input_files[::skip_frames]

            output_files = natsorted(glob.glob(os.path.join(output_dir, '*.tif')))
            if len(input_files) <= len(output_files):
                print(f"Conversion might already be completed or partial for {output_dir}. Continuing...")
                # Optional: Add logic to check and continue incomplete work.

            num_digits = len(str(len(input_files)))

            # Prepare arguments for parallel processing
            args = [(file_name, output_dir, brightness_factor, contrast_factor, num_digits, i)
                    for i, file_name in enumerate(input_files)]

            # Use all available cores
            with Pool(cpu_count()) as pool:
                pool.map(process_single_image, args)


# helper function to plot autocorrelation
def plot_autocorrelation_values(data_path, condition, subcondition, frame_id, lambda_tau, results, fitted_values, intervector_distance_microns):
    output_directory_dfs = os.path.join(data_path, condition, subcondition, "autocorrelation_plots")
    os.makedirs(output_directory_dfs, exist_ok=True)

    plt.figure(figsize=(10, 6))

    x_values = np.arange(len(results)) * intervector_distance_microns * 1E6

    plt.plot(x_values, results, label='Autocorrelation Values', marker='o', linestyle='-', markersize=5)
    plt.plot(x_values, fitted_values, label='Fitted Exponential Decay', linestyle='--', color='red')
    plt.axvline(x=lambda_tau, color='green', linestyle='-.', label=f'Correlation Length = {lambda_tau:.2f} µm')

    plt.xlabel('Scaled Lag (µm)')
    plt.ylabel('Autocorrelation')
    plt.title(f'Autocorrelation Function and Fitted Exponential Decay (Frame {frame_id})')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    # plt.ylim(0, 1.1)

    plt.tight_layout()

    filename = os.path.join(output_directory_dfs, f'autocorrelation_frame_{frame_id}.jpg')
    plt.savefig(filename, dpi=200, format='jpg')
    plt.close()

# helper function to calculate correlation length
def correlation_length(data_frame):
    # Reshaping the data frame to a 2D grid and normalizing
    v = data_frame.pivot(index='y [m]', columns='x [m]', values="velocity magnitude [m/s]").values
    v -= np.mean(v)  # Centering the data

    # FFT to find the power spectrum and compute the autocorrelation
    fft_v = np.fft.fft2(v)
    autocorr = np.fft.ifft2(fft_v * np.conj(fft_v))
    autocorr = np.real(autocorr) / np.max(np.real(autocorr))  # Normalize the autocorrelation

    # Preparing to extract the autocorrelation values along the diagonal
    r_values = min(v.shape) // 2
    results = np.zeros(r_values)
    for r in range(r_values):
        # Properly average over symmetric pairs around the center
        autocorrelation_value = (autocorr[r, r] + autocorr[-r, -r]) / 2
        results[r] = autocorrelation_value

    # Normalize the results to start from 1
    results /= results[0]

    # Exponential decay fitting to extract the correlation length
    def exponential_decay(x, A, B, C):
        return A * np.exp(-x / B) + C

    # Fit parameters and handling potential issues with initial parameter guesses
    try:
        params, _ = curve_fit(exponential_decay, np.arange(len(results)), results, p0=(1, 10, 0), maxfev=5000)
    except RuntimeError:
        # Handle cases where the curve fit does not converge
        params = [np.nan, np.nan, np.nan]  # Use NaN to indicate the fit failed

    A, B, C = params
    fitted_values = exponential_decay(np.arange(r_values), *params)

    # Calculate the correlation length
    intervector_distance_microns = ((data_frame["y [m]"].max() - data_frame["y [m]"].min()) / v.shape[0])
    if B > 0 and A != C:  # Ensure valid values for logarithmic calculation
        lambda_tau = -B * np.log((0.3 - C) / A) * intervector_distance_microns
    else:
        lambda_tau = np.nan  # Return NaN if parameters are not suitable for calculation

    return lambda_tau, results, fitted_values, intervector_distance_microns


# load PIV data from PIVlab into dataframes
def load_piv_data(data_path, condition, subcondition, min_frame=0, max_frame=None, skip_frames=1):
    """
    Processes Particle Image Velocimetry (PIV) data to create a DataFrame that combines mean values, 
    power calculations, and pivot matrices for each feature.

    Args:
        data_path (str): Path to the directory containing PIV data files.
        condition (str): Condition label for the data set.
        subcondition (str): Subcondition label for the data set.
        min_frame (int, optional): Minimum frame index to start processing (inclusive).
        max_frame (int, optional): Maximum frame index to stop processing (exclusive).

    Returns:
        pandas.DataFrame: A DataFrame where each row corresponds to a frame, combining mean values, 
        power calculations, and pivot matrices for each feature.
    """

    input_piv_data = os.path.join(data_path, condition, subcondition, "piv_data", "PIVlab_****.txt")
    
    # Using a for loop instead of list comprehension
    dfs = []
    for file in sorted(glob.glob(input_piv_data))[min_frame:max_frame:skip_frames]:
        df = pd.read_csv(file, skiprows=2).fillna(0).rename(columns={
            "magnitude [m/s]": "velocity magnitude [m/s]",
            "simple shear [1/s]": "shear [1/s]",
            "simple strain [1/s]": "strain [1/s]",
            "Vector type [-]": "data type [-]"
        })
        dfs.append(df)

    return dfs

# store pivlab output as dataframes
def generate_dataframes_from_piv_data(data_path, condition, subcondition, min_frame=0, max_frame=None, skip_frames=1, plot_autocorrelation=True):
    """
    Generates a time series pivot DataFrame from input data.

    Parameters:
    data_path (str): Path to the input data file.
    condition (str): Primary condition for data filtering.
    subcondition (str): Secondary condition for further data filtering.
    min_frame (int, optional): Minimum frame to consider in the analysis. Defaults to 0.
    max_frame (int, optional): Maximum frame to consider in the analysis. If None, considers all frames. Defaults to None.
    plot_autocorrelation (bool, optional): Flag to plot autocorrelation. Defaults to True.
    time_interval (int, optional): Time interval between frames, in seconds. Defaults to 3.

    Returns:
    tuple: A tuple containing two pandas DataFrames. The first is the mean values DataFrame and the second is the pivot matrices DataFrame.
    """
    # Creating output directories
    output_directory_dfs = os.path.join(data_path, condition, subcondition, "dataframes_PIV")
    os.makedirs(output_directory_dfs, exist_ok=True)

    # Load PIV data
    data_frames = load_piv_data(data_path, condition, subcondition, min_frame, max_frame, skip_frames)


    # Calculating mean values with valid vectors only
    mean_values = []
    for frame_id, data_frame in enumerate(data_frames):
        lambda_tau, results, fitted_values, intervector_distance_microns = correlation_length(data_frame)
        if plot_autocorrelation:
            plot_autocorrelation_values(data_path, condition, subcondition, frame_id, lambda_tau * 1E6, results, fitted_values, intervector_distance_microns)
        data_frame["correlation length [m]"] = lambda_tau
        data_frame = data_frame[data_frame["data type [-]"] == 1]
        mean_values.append(data_frame.mean(axis=0))

    # Creating mean DataFrame
    mean_data_frame = pd.DataFrame(mean_values)
    mean_data_frame.reset_index(drop=False, inplace=True)
    mean_data_frame.rename(columns={'index': 'frame'}, inplace=True)

    # Calculate power and add to DataFrame
    volume = 2.5E-9  # µl --> m^3
    viscosity = 1E-3  # mPa*S
    mean_data_frame["power [W]"] = volume * viscosity * (mean_data_frame["velocity magnitude [m/s]"]/mean_data_frame["correlation length [m]"])**2

    # Renaming time column
    # mean_data_frame.rename(columns={'frame': 'time [min]'}, inplace=True)

    # Remove unnecessary columns for the pivot matrices
    # mean_data_frame = mean_data_frame.iloc[:, 5:]

    # Scale time appropriately
    mean_data_frame["frame"] = np.arange(len(mean_data_frame)) 


    # Creating pivot matrices for each feature
    features = data_frames[0].columns[:-1]
    pivot_matrices = {feature: [] for feature in features}

    for data_frame in data_frames:
        temporary_dictionary = {feature: data_frame.pivot(index='y [m]', columns='x [m]', values=feature).values for feature in features}
        for feature in features:
            pivot_matrices[feature].append(temporary_dictionary[feature])

    pivot_data_frame = pd.DataFrame(pivot_matrices)

    # Adjusting column names in mean_data_frame
    mean_data_frame.columns = [f"{column}_mean" if column != "frame" else column for column in mean_data_frame.columns]
    
    # Adding time column to pivot_data_frame
    pivot_data_frame["frame"] = mean_data_frame["frame"].values
    
    # Save DataFrames to CSV
    mean_df_output_path = os.path.join(output_directory_dfs, "mean_values.csv")
    mean_data_frame.to_csv(mean_df_output_path, index=False)

    pivot_df_output_path = os.path.join(output_directory_dfs, "features_matrices.csv")
    pivot_data_frame.to_csv(pivot_df_output_path, index=False)

    # return mean_data_frame, pivot_data_frame, average_values
    return mean_data_frame, pivot_data_frame

# plot the pivlab output as heatmaps
def generate_heatmaps_from_dataframes(df, data_path, condition, subcondition, feature_limits, time_interval=3):
    """
    Generates and saves heatmaps for each feature specified in the feature_limits dictionary.
    Each heatmap is overlaid on a corresponding image and saved to a structured directory.

    Parameters:
    - df (DataFrame): The DataFrame containing the data to plot. Each column represents a feature,
                      and each row represents a frame.
    - data_path (str): Base path for reading source images and saving heatmaps.
    - condition (str): Condition name, used for directory structuring.
    - subcondition (str): Subcondition name, further specifying the directory structure.
    - feature_limits (dict): A dictionary where keys are feature names (column names in df) and
                             values are tuples (vmin, vmax) representing the limits for the heatmap.
    - time_interval (int, optional): Time interval between frames, used for time annotation in the plot title. 
                                     Default is 3.

    The function creates a directory structure under 'data_path' for each feature to store the heatmaps.
    The structure is: data_path/condition/subcondition/heatmaps_PIV/feature_name/.

    Heatmaps are generated for each frame (row in df) and saved as JPEG images.
    """
    
    for feature, limits in feature_limits.items():
        vmin, vmax = limits

        for j in range(len(df)):
            vals = df.iloc[j, df.columns.get_loc(feature)]

            output_directory_heatmaps = os.path.join(data_path, condition, subcondition, "heatmaps_PIV", f"{feature.split()[0]}", f"{feature.split()[0]}_heatmap_{j}.jpg")
            image_files_pattern = f"{data_path}/{condition}/{subcondition}/piv_movie_converted/converted_image_****.tif"
            image_files = sorted(glob.glob(image_files_pattern))[j]
            image = Image.open(image_files)

            plt.figure(figsize=(10, 6))
            plt.imshow(image, cmap=None, extent=[-2762/2, 2762/2, -2762/2, 2762/2]) # piv image
            im = plt.imshow(vals, cmap='inferno', origin='upper', alpha=0.7, extent=[-2762/2, 2762/2, -2762/2, 2762/2], vmin=vmin, vmax=vmax) # heatmap
            plt.xlabel('x [um]')
            plt.ylabel('y [um]')
            cbar = plt.colorbar(im)
            cbar.set_label(feature)
            time = df.iloc[j, -1]
            plt.title(f"PIV - {feature}  ||  time: {int(time * time_interval/60)} min")

            os.makedirs(os.path.dirname(output_directory_heatmaps), exist_ok=True)
            plt.savefig(output_directory_heatmaps, format='jpg', dpi=250)
            plt.close()



# create a movie from the processed images -- general function
def create_movies_PIV(data_path, condition, subcondition, channel, movie_type, frame_rate, feature_limits=None, max_frame=None):
    """
    Creates video files from processed and annotated images stored in a specified directory.

    Args:
    - data_path (str): Base path where the annotated images are stored.
    - condition (str): Condition under which the annotated images are stored.
    - subcondition (str): Subcondition under which the annotated images are stored.
    - channel (str): The specific channel being processed ('cy5' or 'gfp').
    - movie_type (str): Type of movie to create ('single', 'grid', or 'PIV').
    - feature_limits (dict, optional): Dictionary specifying the limits for each feature (only for 'PIV' movie type).
    - frame_rate (int, optional): Frame rate for the output video. Defaults to 120.
    - max_frame (int, optional): Maximum number of frames to be included in the video. If None, all frames are included.
    """

    plots_dir = f"{data_path}/{condition}/{subcondition}/heatmaps_PIV/"
    for feature in feature_limits.keys():
        feature_name_for_file = feature.split()[0]
        heatmap_dir = os.path.join(data_path, condition, subcondition, "heatmaps_PIV", f"{feature.split()[0]}", f"{feature.split()[0]}_heatmap_****.jpg")
        image_files = natsorted(glob.glob(heatmap_dir))

        if not image_files:
            continue

        # Limit the number of files if max_frame is specified
        image_files = image_files[:max_frame] if max_frame is not None else image_files

        # Get the resolution of the first image (assuming all images are the same size)
        first_image = cv2.imread(image_files[0])
        video_resolution = (first_image.shape[1], first_image.shape[0])  # Width x Height

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter(f'{plots_dir}{feature_name_for_file}.avi', fourcc, frame_rate, video_resolution)

        for file in image_files:
            img = cv2.imread(file)
            out.write(img)  # Write the image as is, without resizing

        out.release()
        return

    if not image_files:
        print("No images found for video creation.")
        return

    # Limit the number of files if max_frame is specified
    image_files = image_files[:max_frame] if max_frame is not None else image_files

    # Get the resolution of the first image (assuming all images are the same size)
    first_image = cv2.imread(image_files[0])
    video_resolution = (first_image.shape[1], first_image.shape[0])  # Width x Height

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(out_path, fourcc, frame_rate, video_resolution)

    for file_path in image_files:
        img = cv2.imread(file_path)
        out.write(img)  # Write the image frame to the video

    out.release()
    print(f"Video saved to {out_path}")

# turn heatmaps into movies 
def process_piv_data(data_path, conditions, subconditions, feature_limits, time_intervals, min_frame=0, max_frame=None, skip_frames=1, plot_autocorrelation=True, frame_rate=120, heatmaps=True):
    """Process PIV data for all conditions and subconditions, then average and save results.

    Args:
        data_path (str): Base directory for PIV data and output.
        conditions (list): List of conditions.
        subconditions (list): List of subconditions.
        feature_limits (dict): Dictionary of feature limits.
        time_intervals (list): List of time intervals matching the conditions.
        min_frame (int, optional): Minimum frame number to process. Defaults to 0.
        max_frame (int, optional): Maximum frame number to process. Defaults to None.
        skip_frames (int, optional): Number of frames to skip between processed frames. Defaults to 1.
        plot_autocorrelation (bool, optional): Whether to plot autocorrelation. Defaults to True.
        frame_rate (int, optional): Frame rate for the movies. Defaults to 120.
    """
    for i, condition in enumerate(conditions):
        time_interval = time_intervals[i] * skip_frames
        results = []
        for subcondition in subconditions:
            m, p = generate_dataframes_from_piv_data(data_path, condition, subcondition, min_frame, max_frame, skip_frames, plot_autocorrelation)
            results.append(m)

            if heatmaps == True:
                generate_heatmaps_from_dataframes(p, data_path, condition, subcondition, feature_limits, time_interval)
                create_movies_PIV(data_path, condition, subcondition, channel=None, movie_type='PIV', feature_limits=feature_limits, frame_rate=frame_rate, max_frame=max_frame)

        # Averaging and saving the results for the current condition
        save_path = os.path.join(data_path, condition, 'averaged')
        average_df = sum(results) / len(results)
        
        os.makedirs(save_path, exist_ok=True)  # Ensure the directory exists
        average_df.to_csv(os.path.join(save_path, f"{condition}_average.csv"))


In [8]:
# Example usage
v = 2E-6
velocity_limits = (0, v)
other_limits = (-0.0005, 0.0005)


# velocity_limits = (None, None)
# other_limits = (None, None)


feature_limits = {
    'u [m/s]': (-v, v), 
    'v [m/s]': (-v, v), 
    'data type [-]': (None, None),
    'velocity magnitude [m/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    'dcev [1]': (0, 250),
    'shear [1/s]': other_limits,
    'strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}

skip_frames = 64
time_interval_list = [120] * len(conditions)  # time intervals in seconds between frames for each condition


# Convert images to the right size
convert_images(data_path, conditions, subconditions, max_frame=None, brightness_factor=1, contrast_factor=1, skip_frames=skip_frames)
# Process PIV data
process_piv_data(data_path, conditions, subconditions, feature_limits, time_interval_list, min_frame=0, max_frame=None, skip_frames=skip_frames, plot_autocorrelation=True, frame_rate=1, heatmaps=True)


In [7]:
import os
import shutil

def delete_generated_outputs(data_path, conditions, subconditions, delete_images=True, delete_autocorrelations=True, delete_heatmaps=True, delete_movies=True, delete_dataframes=True):
    """
    Deletes generated output files and directories for the specified conditions and subconditions.

    Args:
    - data_path (str): Base directory for the outputs.
    - conditions (list): List of conditions.
    - subconditions (list): List of subconditions.
    - delete_images (bool): Flag to delete converted images. Defaults to True.
    - delete_autocorrelations (bool): Flag to delete autocorrelation plots. Defaults to True.
    - delete_heatmaps (bool): Flag to delete heatmaps. Defaults to True.
    - delete_movies (bool): Flag to delete movies. Defaults to True.
    - delete_dataframes (bool): Flag to delete dataframes. Defaults to True.
    """
    for condition in conditions:
        for subcondition in subconditions:
            if delete_images:
                converted_images_dir = os.path.join(data_path, condition, subcondition, "piv_movie_converted")
                if os.path.exists(converted_images_dir):
                    shutil.rmtree(converted_images_dir)
                    print(f"Deleted directory: {converted_images_dir}")

            if delete_autocorrelations:
                autocorrelation_dir = os.path.join(data_path, condition, subcondition, "autocorrelation_plots")
                if os.path.exists(autocorrelation_dir):
                    shutil.rmtree(autocorrelation_dir)
                    print(f"Deleted directory: {autocorrelation_dir}")

            if delete_heatmaps:
                heatmaps_dir = os.path.join(data_path, condition, subcondition, "heatmaps_PIV")
                if os.path.exists(heatmaps_dir):
                    shutil.rmtree(heatmaps_dir)
                    print(f"Deleted directory: {heatmaps_dir}")

            if delete_movies:
                single_movies_dir = os.path.join(data_path, f"single_movies_{subcondition}")
                if os.path.exists(single_movies_dir):
                    shutil.rmtree(single_movies_dir)
                    print(f"Deleted directory: {single_movies_dir}")

                grid_movies_dir = os.path.join(data_path, f"grid_heatmaps_{subcondition}")
                if os.path.exists(grid_movies_dir):
                    shutil.rmtree(grid_movies_dir)
                    print(f"Deleted directory: {grid_movies_dir}")

                plots_dir = os.path.join(data_path, condition, subcondition, "heatmaps_PIV")
                for root, dirs, files in os.walk(plots_dir):
                    for file in files:
                        if file.endswith(".avi"):
                            os.remove(os.path.join(root, file))
                            print(f"Deleted file: {os.path.join(root, file)}")

            if delete_dataframes:
                dataframes_dir = os.path.join(data_path, condition, subcondition, "dataframes_PIV")
                if os.path.exists(dataframes_dir):
                    shutil.rmtree(dataframes_dir)
                    print(f"Deleted directory: {dataframes_dir}")

# Example usage:
delete_generated_outputs(data_path, conditions, subconditions)


Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/kif3/Rep1/piv_movie_converted
Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/kif3/Rep1/autocorrelation_plots
Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/kif3/Rep1/heatmaps_PIV
Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/kif3/Rep1/dataframes_PIV
Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/kif3/Rep2/piv_movie_converted
Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/kif3/Rep2/autocorrelation_plots
Deleted directory: ../../../../Thomson Lab Dropbox/David Larios/activedrops/1&2-Mechanism&Phases/Kif3-3reps-3sint-piv/ki